In [51]:
from keplergl import KeplerGl
import pandas as pd
pd.options.mode.chained_assignment = None
import math
import numpy as np
import datetime
from time import sleep
from tqdm import tqdm

# show all rows
pd.set_option('display.max_rows', None)

# import Data
df01 = pd.read_csv('june_to_august2019_with_weather.csv',
                    parse_dates=['start_date'])

df01 = df01[['bike_id','start_date','start_lng','start_lat','end_date','end_lng','end_lat']]

# set Datetime Format
df01['start_date']=pd.to_datetime(df01['start_date'])
df01['end_date']=pd.to_datetime(df01['end_date'])
df01['bike_id'] = pd.to_numeric(df01['bike_id'])

#cut 
df01 = df01.truncate(after = 1000)

In [52]:
# Dict out of DataFrames of Trips per Bike

# init DFs for every bike
myDFs = {}

for ind in tqdm(df01.index):
    if df01['bike_id'][ind] not in myDFs:
        myDFs[df01['bike_id'][ind]] = pd.DataFrame(columns = 
                                                   ['bike_id','start_date','start_lng','start_lat','end_date','end_lng','end_lat'])
        
#myDFs

100%|█████████████████████████████████████| 1001/1001 [00:00<00:00, 2003.65it/s]


In [53]:
# Add all rentals to their bike_id DF
for ind in tqdm(df01.index):
    if df01['bike_id'][ind] in myDFs:
        myDFs[df01['bike_id'][ind]].loc[df01.index[ind]] = df01.iloc[ind]
#myDFs

100%|██████████████████████████████████████| 1001/1001 [00:03<00:00, 320.77it/s]


In [54]:
# sort every DF
for df in myDFs.values():
    df.sort_values(by=['start_date'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    #print(df)

In [55]:
print(myDFs)

{72246:   bike_id          start_date  start_lng  start_lat            end_date  \
0   72246 2019-06-01 23:15:02  12.365141  51.341253 2019-06-01 23:25:31   

     end_lng    end_lat  
0  12.371609  51.354213  , 41674:   bike_id          start_date  start_lng  start_lat            end_date  \
0   41674 2019-06-05 16:37:01  12.365500  51.341689 2019-06-05 16:41:39   
1   41674 2019-07-28 01:02:25  12.371979  51.331609 2019-07-28 01:17:57   
2   41674 2019-07-28 15:28:05  12.334168  51.337240 2019-07-28 15:29:44   
3   41674 2019-08-06 20:01:43  12.423901  51.305071 2019-08-21 14:27:53   

     end_lng    end_lat  
0  12.379628  51.344577  
1  12.334542  51.336840  
2  12.333877  51.336827  
3  12.423901  51.305071  , 93381:   bike_id          start_date  start_lng  start_lat            end_date  \
0   93381 2019-06-06 00:53:22  12.381495  51.343813 2019-06-06 00:58:41   
1   93381 2019-07-26 08:20:41  12.378853  51.321671 2019-07-26 08:35:14   
2   93381 2019-07-26 17:59:45  12.336922  

In [56]:
# DF of all points with stoodtime
df_points = pd.DataFrame(columns = ['bike_id','lng','lat','timestood','time_start','time_end'])

for df in tqdm(myDFs.values()):
    df = df.reset_index(drop=True)
    for ind in df.index[1:]:
        if 0 :
            print("nan")
            continue
        try:
            #print((df['start_date'][ind] - df['end_date'][ind - 1]).seconds / 60)
            timestood = (df['start_date'][ind] - df['end_date'][ind - 1]).seconds / 60
            if timestood == 0:
                break
        except:
            print("Timestood = 0 ----- " + str(df['start_date'][ind]) + "  -  " + str(df['end_date'][ind - 1]))
            timestood = 0
        dict = {'bike_id': df['bike_id'][ind] ,'lng': df['start_lng'][ind] , 'lat': df['start_lat'][ind] , 'timestood': timestood, 'time_start': df['end_date'][ind - 1], 'time_end': df['start_date'][ind]}
        df_points = df_points.append(dict , ignore_index = True)

100%|████████████████████████████████████████| 567/567 [00:01<00:00, 433.11it/s]


In [57]:

df_points = df_points.dropna()
df_points['time_start'] = df_points['time_start'].astype(str)
df_points['time_end'] = df_points['time_end'].astype(str)
df_points['bike_id'] = df_points['bike_id'].astype(str)
df_points['timestood'] = df_points['timestood'].astype(str)
df_points['lng'] = df_points['lng'].astype(float)
df_points['lat'] = df_points['lat'].astype(float)
df_points

,bike_id,lng,lat,timestood,time_start,time_end
0,41674,12.371979,51.331609,500.76666666666665,2019-06-05 16:41:39,2019-07-28 01:02:25
1,41674,12.334168,51.337240,850.1333333333333,2019-07-28 01:17:57,2019-07-28 15:28:05
2,41674,12.423901,51.305071,271.98333333333335,2019-07-28 15:29:44,2019-08-06 20:01:43
3,93381,12.378853,51.321671,442.0,2019-06-06 00:58:41,2019-07-26 08:20:41
4,93381,12.336922,51.324615,564.5166666666667,2019-07-26 08:35:14,2019-07-26 17:59:45
5,93381,12.336922,51.324615,391.6166666666667,2019-07-29 10:37:31,2019-07-29 17:09:08
6,72319,12.373557,51.323973,1287.7333333333333,2019-06-07 21:21:48,2019-07-30 18:49:32
7,93444,12.345892,51.366911,24.583333333333332,2019-06-09 18:55:07,2019-08-26 19:19:42
8,93369,12.350821,51.332196,1423.4333333333334,2019-06-10 11:12:07,2019-06-28 10:55:33
9,72278,12.341799,51.329827,251.58333333333334,2019-06-10 15:55:32,2019-08-28 20:07:07


In [58]:
heatmap = KeplerGl()
heatmap.add_data(data=df_points, name = 'points')
heatmap

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'points': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 16, 17, 18, 19, 20, 21, 22, 23, …

In [50]:
heatmap.save_to_html(file_name='heatmap_all.html')

Map saved to heatmap_all.html!


In [59]:
df_points.to_csv('df_points.csv', index=False)

In [11]:
df_points.dtypes

bike_id        object
lng           float64
lat           float64
timestood     float64
time_start     object
time_end       object
dtype: object

In [12]:
print(np.datetime64('2015-05-17 06:01:23') < np.datetime64('0000-00-00 06:0:23'))

ValueError: Month out of range in datetime string "0000-00-00 06:0:23"

In [ ]:
aa = np.datetime64('2015-05-17 06:01:23').time().strftime('%H:%M:%S')

In [45]:
Timestamp(np.datetime64('2015-05-17 06:01:23'))

NameError: name 'Timestamp' is not defined

In [ ]:
for ind in df_points.index:
    if df_points['time_start'][ind] > 